# Object changes

Analyse two snapshots for the aggregates that changed.

In [20]:
import glob
import json
import ipaddress
import pathlib
import os

import pandas as pd

In [58]:
chunks = []

base_dir = pathlib.Path("~/Downloads").expanduser()

for path in base_dir.glob("*/output/rpki-client.json"):
    with path.open("r") as f:
        chunk = pd.DataFrame(json.load(f)["roas"])
        chunk['file'] = path.relative_to(base_dir)
        # And add address family
        chunk['af'] = chunk.prefix.map(lambda p: ipaddress.ip_network(p).version)
        # And prefix length (to chart later)
        chunk['prefix_length'] = chunk.prefix.map(lambda p: ipaddress.ip_network(p).prefixlen)

        chunks.append(chunk)
        
df = pd.concat(chunks)

In [59]:
df.file.unique()

array([PosixPath('rpki-20231002T030544Z/output/rpki-client.json'),
       PosixPath('rpki-20231002T050457Z/output/rpki-client.json')],
      dtype=object)

In [31]:
df_first = chunks[0].drop(columns="file")
df_second = chunks[1].drop(columns="file")

In [51]:
df_joined = df_first.set_index(["asn", "prefix", "maxLength"]).join(df_second.set_index(["asn", "prefix", "maxLength"]), lsuffix="_before", rsuffix="_after", how="left")
df_joined.sample(5)

,,,ta_before,expires_before,af_before,prefix_length_before,ta_after,expires_after,af_after,prefix_length_after
asn,prefix,maxLength,,,,,,,,
131627,106.105.252.0/22,24,apnic,1696298297,4,22,apnic,1.696298e+09,4.0,22.0
38496,116.66.200.0/21,21,apnic,1696337468,4,21,apnic,1.696337e+09,4.0,21.0
149015,103.176.86.0/23,24,apnic,1696728023,4,23,apnic,1.696728e+09,4.0,23.0
41722,2a03:21c0::/32,32,ripe,1696284060,6,32,ripe,1.696284e+09,6.0,32.0
23881,203.135.139.0/24,24,apnic,1696702466,4,24,apnic,1.696702e+09,4.0,24.0


In [52]:
df_joined[df_joined.prefix_length_after.isna() & df_joined.ta_before=="arin"]

,,,ta_before,expires_before,af_before,prefix_length_before,ta_after,expires_after,af_after,prefix_length_after
asn,prefix,maxLength,,,,,,,,


In [56]:
df_joined[(df_joined.ta_before=="arin") & df_joined.prefix_length_after.isna()]

ta_before  expires_before  af_before  \
asn    prefix             maxLength                                        
2      50.236.231.0/24    24             arin      1696219200          4   
5607   184.120.0.0/21     21             arin      1696219200          4   
       184.120.32.0/21    21             arin      1696219200          4   
       184.120.32.0/19    19             arin      1696219200          4   
400098 199.36.120.0/24    24             arin      1696343712          4   
14618  2602:fcee:400::/48 48             arin      1696338000          6   
16509  2602:fcee:400::/48 48             arin      1696338000          6   
14618  2602:fcee:404::/48 48             arin      1696338000          6   
16509  2602:fcee:404::/48 48             arin      1696338000          6   
14618  2602:fcee:408::/48 48             arin      1696338000          6   
16509  2602:fcee:408::/48 48             arin      1696338000          6   
14618  2602:fcee:ff4::/48 48             arin      1696338000          6   
16509  2602:fcee:ff4::/48 48             arin      1696338000          6   
14618  2602:fcee:ff5::/48 48             arin      1696338000          6   
16509  2602:fcee:ff5::/48 48             arin      1696338000          6   
14618  2602:fcee:ff6::/48 48             arin      1696338000          6   
16509  2602:fcee:ff6::/48 48             arin      1696338000          6   

                                     prefix_length_before ta_after  \
asn    prefix             maxLength                                  
2      50.236.231.0/24    24                           24      NaN   
5607   184.120.0.0/21     21                           21      NaN   
       184.120.32.0/21    21                           21      NaN   
       184.120.32.0/19    19                           19      NaN   
400098 199.36.120.0/24    24                           24      NaN   
14618  2602:fcee:400::/48 48                           48      NaN   
16509  2602:fcee:400::/48 48                           48      NaN   
14618  2602:fcee:404::/48 48                           48      NaN   
16509  2602:fcee:404::/48 48                           48      NaN   
14618  2602:fcee:408::/48 48                           48      NaN   
16509  2602:fcee:408::/48 48                           48      NaN   
14618  2602:fcee:ff4::/48 48                           48      NaN   
16509  2602:fcee:ff4::/48 48                           48      NaN   
14618  2602:fcee:ff5::/48 48                           48      NaN   
16509  2602:fcee:ff5::/48 48                           48      NaN   
14618  2602:fcee:ff6::/48 48                           48      NaN   
16509  2602:fcee:ff6::/48 48                           48      NaN   

                                     expires_after  af_after  \
asn    prefix             maxLength                            
2      50.236.231.0/24    24                   NaN       NaN   
5607   184.120.0.0/21     21                   NaN       NaN   
       184.120.32.0/21    21                   NaN       NaN   
       184.120.32.0/19    19                   NaN       NaN   
400098 199.36.120.0/24    24                   NaN       NaN   
14618  2602:fcee:400::/48 48                   NaN       NaN   
16509  2602:fcee:400::/48 48                   NaN       NaN   
14618  2602:fcee:404::/48 48                   NaN       NaN   
16509  2602:fcee:404::/48 48                   NaN       NaN   
14618  2602:fcee:408::/48 48                   NaN       NaN   
16509  2602:fcee:408::/48 48                   NaN       NaN   
14618  2602:fcee:ff4::/48 48                   NaN       NaN   
16509  2602:fcee:ff4::/48 48                   NaN       NaN   
14618  2602:fcee:ff5::/48 48                   NaN       NaN   
16509  2602:fcee:ff5::/48 48                   NaN       NaN   
14618  2602:fcee:ff6::/48 48                   NaN       NaN   
16509  2602:fcee:ff6::/48 48                   NaN       NaN   

                                     prefix